# Eksperiment prompt

## Set Up Module dan Environment

In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import TypedDict, List, Optional, Literal, Annotated, Any
from langchain_google_genai import ChatGoogleGenerativeAI
import traceback

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

from enum import Enum
import operator
import re
from copy import deepcopy
from datetime import date, datetime, timedelta, time, timezone
from zoneinfo import ZoneInfo
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text, Date, DateTime, ForeignKey, UniqueConstraint, func, Boolean, select, insert, update, and_, between
from langchain_core.messages import (
    AnyMessage, HumanMessage, ToolMessage, AIMessage, SystemMessage
)

from zoneinfo import ZoneInfo

In [2]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData

load_dotenv()  # ini akan memuat isi file .env ke environment

def setup_environment():
    """Memuat semua environment variables, menyimpannya ke os.environ, dan menampilkan nilai TERMASKED."""

    def mask_value(val: str, visible_fraction: float = 0.5) -> str:
        if val is None:
            return ""
        s = str(val)
        n = len(s)
        if n <= 4:
            return "*" * n
        visible = max(1, int(n * visible_fraction))
        return s[:visible] + "*" * (n - visible)

    env_vars = [
        "GOOGLE_API_KEY",
        "LANGSMITH_API_KEY",
        "LANGSMITH_TRACING",
        "LANGSMITH_ENDPOINT",
        "LANGSMITH_PROJECT",
        "DATABASE_URL"
    ]
    for var in env_vars:
        value = os.getenv(var)
        if not value:
            raise RuntimeError(
                f"{var} not found in environment. Set it in .env or export it."
            )
        os.environ[var] = value
        print(f"{var} Terload! Value: {mask_value(value)}")

setup_environment()

DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
metadata = MetaData()

GOOGLE_API_KEY Terload! Value: AIzaSyD6a8iF0_8-Ztw********************
LANGSMITH_API_KEY Terload! Value: lsv2_pt_84b8750ca1c74b349**************************
LANGSMITH_TRACING Terload! Value: ****
LANGSMITH_ENDPOINT Terload! Value: https://api.smi****************
LANGSMITH_PROJECT Terload! Value: TiketaAgentsE**************
DATABASE_URL Terload! Value: postgresql+psycopg://postgres:*******************************


In [3]:
SEAT_MAP: list[list[str | None]] = [
    # A
    [
        "A1",
        "A2",
        "A3",
        "A4",
        "A5",
        "A6",
        "A7",
        "A8",
        "A9",
        None,
        "A10",
        "A11",
        "A12",
        "A13",
        "A14",
        "A15",
        "A16",
        "A17",
        "A18",
    ],
    # B
    [
        "B1",
        "B2",
        "B3",
        "B4",
        "B5",
        "B6",
        "B7",
        "B8",
        "B9",
        None,
        "B10",
        "B11",
        "B12",
        "B13",
        "B14",
        "B15",
        "B16",
        "B17",
        "B18",
    ],
    # C
    [
        "C1",
        "C2",
        "C3",
        "C4",
        "C5",
        "C6",
        "C7",
        "C8",
        "C9",
        None,
        "C10",
        "C11",
        "C12",
        "C13",
        "C14",
        "C15",
        "C16",
        "C17",
        "C18",
    ],
    # D
    [
        "D1",
        "D2",
        "D3",
        "D4",
        "D5",
        "D6",
        "D7",
        "D8",
        "D9",
        None,
        "D10",
        "D11",
        "D12",
        "D13",
        "D14",
        "D15",
        "D16",
        "D17",
        "D18",
    ],
    # E
    [
        "E1",
        "E2",
        "E3",
        "E4",
        "E5",
        "E6",
        "E7",
        "E8",
        "E9",
        None,
        "E10",
        "E11",
        "E12",
        "E13",
        "E14",
        "E15",
        "E16",
        "E17",
        "E18",
    ],
    # F
    [
        "F1",
        "F2",
        "F3",
        "F4",
        "F5",
        "F6",
        "F7",
        "F8",
        "F9",
        None,
        "F10",
        "F11",
        "F12",
        "F13",
        "F14",
        "F15",
        "F16",
        "F17",
        "F18",
    ],
    # G
    [
        "G1",
        "G2",
        "G3",
        "G4",
        "G5",
        "G6",
        "G7",
        "G8",
        "G9",
        None,
        "G10",
        "G11",
        "G12",
        "G13",
        "G14",
        "G15",
        "G16",
        "G17",
        "G18",
    ],
    # H
    [
        "H1",
        "H2",
        "H3",
        "H4",
        "H5",
        "H6",
        "H7",
        "H8",
        "H9",
        None,
        "H10",
        "H11",
        "H12",
        "H13",
        "H14",
        "H15",
        "H16",
        "H17",
        "H18",
    ],
    # I
    [
        "I1",
        "I2",
        "I3",
        "I4",
        "I5",
        "I6",
        "I7",
        "I8",
        "I9",
        None,
        "I10",
        "I11",
        "I12",
        "I13",
        "I14",
        "I15",
        "I16",
        "I17",
        "I18",
    ],
    # J
    [
        "J1",
        "J2",
        "J3",
        "J4",
        "J5",
        "J6",
        "J7",
        "J8",
        "J9",
        None,
        "J10",
        "J11",
        "J12",
        "J13",
        "J14",
        "J15",
        "J16",
        "J17",
        "J18",
    ],
    # K (kosong lorong)
    [None] * 19,
    # L (lebih sempit)
    [
        "L1",
        "L2",
        "L3",
        "L4",
        "L5",
        "L6",
        "L7",
        "L8",
        "L9",
        None,
        "L10",
        "L11",
        "L12",
        "L13",
        "L14",
        "L15",
        "L16",
        "L17",
        "L18",
    ],
    # M (lebih sempit lagi)
    [
        "M1",
        "M2",
        "M3",
        "M4",
        "M5",
        "M6",
        "M7",
        "M8",
        "M9",
        None,
        "M10",
        "M11",
        "M12",
        "M13",
        "M14",
        "M15",
        "M16",
        "M17",
        "M18",
    ],
]
ALL_VALID_SEATS = {seat for row in SEAT_MAP for seat in row if seat}

In [4]:
# Definisikan tabel database ke bahasa Python menggunakan SQLAlchemy
genres_table = Table("genres", metadata, Column("id", Integer, primary_key=True, autoincrement=True), Column("name", String(120), nullable=False, unique=True), Column("created_at", DateTime, default=func.now()))
movies_table = Table("movies", metadata, Column("id", Integer, primary_key=True, autoincrement=True), Column("title", String(255), nullable=False), Column("description", Text), Column("studio_number", Integer, nullable=False, unique=True), Column("release_date", Date), Column("created_at", DateTime, default=func.now()))
movie_genres_table = Table("movie_genres", metadata, Column("movie_id", Integer, ForeignKey("movies.id"), primary_key=True), Column("genre_id", Integer, ForeignKey("genres.id"), primary_key=True))
showtimes_table = Table("showtimes", metadata, Column("id", Integer, primary_key=True, autoincrement=True), Column("movie_id", Integer, ForeignKey("movies.id"), nullable=False), Column("time", DateTime, nullable=False), Column("is_archived", Boolean, nullable=False, default=False), Column("created_at", DateTime, default=func.now()))
bookings_table = Table("bookings", metadata, Column("id", Integer, primary_key=True, autoincrement=True), Column("user", String(255), nullable=False), Column("seat", String(10), nullable=False), Column("showtime_id", Integer, ForeignKey("showtimes.id"), nullable=False), Column("created_at", DateTime, default=func.now()), UniqueConstraint("showtime_id", "seat", name="uq_booking_showtime_seat"))

In [5]:
TARGET_TZ = ZoneInfo("Asia/Jakarta")
UTC_TZ = ZoneInfo("UTC")

def to_utc_range_naive(date_local_str: str) -> tuple[datetime, datetime]:
    """Mengambil string 'YYYY-MM-DD' WIB, mengembalikan rentang Naive UTC."""
    try:
        local_date = datetime.strptime(date_local_str, "%Y-%m-%d").date()
    except ValueError:
        raise ValueError("Format tanggal salah. Gunakan YYYY-MM-DD.")
        
    start_local_aware = datetime(local_date.year, local_date.month, local_date.day, 0, 0, 0, tzinfo=TARGET_TZ)
    end_local_aware = datetime(local_date.year, local_date.month, local_date.day, 23, 59, 59, tzinfo=TARGET_TZ)
    
    start_utc_aware = start_local_aware.astimezone(UTC_TZ)
    end_utc_aware = end_local_aware.astimezone(UTC_TZ)
    
    return start_utc_aware.replace(tzinfo=None), end_utc_aware.replace(tzinfo=None)

def from_db_utc_naive_to_local_display(utc_dt_naive: datetime) -> str:
    """Mengambil Naive UTC dari DB, mengembalikan string 'HH:MM WIB'."""
    utc_aware = utc_dt_naive.replace(tzinfo=UTC_TZ)
    local_aware = utc_aware.astimezone(TARGET_TZ)
    return local_aware.strftime("%H:%M WIB")

## StateGraph

In [6]:
class TicketAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]
    intent: Literal["browsing", "booking", "confirmation", "other"]

    all_movies_list: List[dict] 

    # --- SLOT FORMULIR (Nama Lebih Jelas) ---
    current_movie_id: Optional[int]         # ID Film yang sedang diproses
    current_showtime_id: Optional[int]      # ID Jadwal yang dipilih untuk film di atas
    selected_seats: Optional[List[str]]     # Daftar Kursi yang dipilih untuk jadwal di atas
    customer_name: Optional[str]            # Nama pemesan (lebih jelas dari 'user')

    # --- KONTEKS SESAAT UNTUK SELEKTOR ---
    context_showtimes: Optional[List[dict]] # Daftar jadwal yg ditawarkan utk current_movie_id
    context_seats: Optional[List[str]]     # Daftar kursi yg ditawarkan utk current_showtime_id

    # --- META-DATA ---
    confirmation_data: Optional[dict] 
    last_error: Optional[str]


## Tools 

In [7]:
@tool #SUCCESS
def get_showtimes(movie_id: int, date_local: str) -> List[dict]:    
    """
    MENGAMBIL jadwal tayang untuk 1 film pada 1 tanggal LOKAL (WIB).
    'date_local' HARUS dalam format 'YYYY-MM-DD'.
    """
    print(f"    > TOOL: get_showtimes(movie_id={movie_id}, date_local='{date_local}')")
    try:
        start_utc, end_utc = to_utc_range_naive(date_local)
    except ValueError as e:
        print(f"    > ERROR di get_showtimes: {e}")
        # Kembalikan pesan error yang jelas agar LLM tahu
        return [{"error": f"Format tanggal salah: {e}. Minta format YYYY-MM-DD."}] 

    stmt = select(showtimes_table.c.id, showtimes_table.c.time).where(
        and_(
            showtimes_table.c.movie_id == movie_id,
            showtimes_table.c.time.between(start_utc, end_utc),
            showtimes_table.c.is_archived.is_(False)
        )
    ).order_by(showtimes_table.c.time)
    
    try:
        with engine.connect() as conn:
            results = conn.execute(stmt).fetchall()
            if not results:
                return [{"message": "Tidak ada jadwal ditemukan untuk tanggal tersebut."}]
            
            showtimes_data = [
                {
                    "showtime_id": row.id,
                    "time_display": from_db_utc_naive_to_local_display(row.time)
                }
                for row in results
            ]
            print(f"    > TOOL get_showtimes: Menemukan {len(showtimes_data)} jadwal.")
            return showtimes_data
    except Exception as e:
        print(f"    > ERROR DB di get_showtimes: {e}")
        return [{"error": f"Gagal mengambil jadwal dari database: {e}"}]

# Definisikan skema output tool agar lebih jelas (opsional tapi bagus)
class SeatAvailabilityInfo(BaseModel):
    count_available: int = Field(description="Jumlah kursi yang tersedia.")
    count_booked: int = Field(description="Jumlah kursi yang sudah terisi.")
    summary_for_llm: str = Field(description="Ringkasan tekstual kursi (tersedia & terisi) untuk prompt LLM.")
    available_list: List[str] = Field(description="Daftar lengkap kursi tersedia.")
    booked_list: List[str] = Field(description="Daftar lengkap kursi terisi.") # Tambahkan ini jika perlu

@tool
def get_available_seats(showtime_id: int) -> SeatAvailabilityInfo: # <-- Ubah tipe return
    """
    MENGAMBIL ringkasan (tersedia & terisi) dan daftar lengkap kursi tersedia 
    untuk 1 jadwal.
    """
    print(f"    > TOOL: get_available_seats(showtime_id={showtime_id})")
    stmt = select(bookings_table.c.seat).where(
        bookings_table.c.showtime_id == showtime_id
    )
    
    # Inisialisasi default jika terjadi error
    default_error_return = SeatAvailabilityInfo(
        count_available=0, 
        count_booked=0,
        summary_for_llm="Error: Gagal mengambil data kursi.",
        available_list=[],
        booked_list=[] 
    )

    try:
        booked_seats: Set[str] # Type hint untuk kejelasan
        with engine.connect() as conn:
            booked_seats = {row.seat for row in conn.execute(stmt).fetchall()}

        available = sorted([seat for seat in ALL_VALID_SEATS if seat not in booked_seats])
        count_available = len(available)
        count_booked = len(booked_seats)
        booked_list_sorted = sorted(list(booked_seats)) # Urutkan kursi terisi

        print(f"    > TOOL get_available_seats: dari 216 kursi, Menemukan {count_available} tersedia, {count_booked} sudah terisi.")

        # --- Buat ringkasan (summary) BARU ---
        summary_lines = []
        summary_lines.append(f"{count_available} kursi tersedia.")
        
        # Tambahkan info kursi terisi (maks 10)
        if count_booked == 0:
            summary_lines.append("Belum ada kursi yang terisi.")
        else:
            booked_display_limit = 10
            booked_info = f"{count_booked} kursi terisi:"
            if count_booked <= booked_display_limit:
                booked_info += f" {', '.join(booked_list_sorted)}"
            else: # Jika > 10, tampilkan 10 pertama + "..."
                booked_info += f" {', '.join(booked_list_sorted[:booked_display_limit])}..."
            summary_lines.append(booked_info)

        # (Opsional: Tambahkan contoh kursi tersedia jika masih relevan)
        if 0 < count_available <= 20: 
             summary_lines.append(f"Kursi tersedia: {', '.join(available)}")
        elif count_available > 20:
             mid_index = count_available // 2
             examples = sorted(list(set([available[0], available[mid_index], available[-1]])))
             summary_lines.append(f"Contoh kursi tersedia: {examples[0]} ... {examples[-1]}")
             
        summary_str = " ".join(summary_lines) # Gabungkan jadi satu string
        # --- Akhir pembuatan summary ---

        # Kembalikan dalam format dictionary/Pydantic
        return SeatAvailabilityInfo(
            count_available=count_available,
            count_booked=count_booked,
            summary_for_llm=summary_str, # Gunakan nama field baru
            available_list=available,
            booked_list=booked_list_sorted # Sertakan list lengkap terisi
        )

    except Exception as e:
        print(f"    > ERROR DB di get_available_seats: {e}")
        # Kembalikan ringkasan error
        return SeatAvailabilityInfo(
            count_available=0,
            count_booked=0, # Asumsikan 0 jika error
            summary_for_llm=f"Error: Gagal mengambil data kursi: {e}",
            available_list=[],
            booked_list=[] # Asumsikan kosong jika error
        )

# Tool 3: Bertanya ke User (Meta-Tool, Sudah Benar)
class AskUserSchema(BaseModel):
    question: str = Field(description="Pertanyaan yang jelas dan spesifik untuk diajukan ke user.")

@tool(args_schema=AskUserSchema)
def ask_user(question: str) -> str:
    """
    Gunakan tool ini JIKA kamu butuh informasi LEBIH LANJUT dari user 
    untuk mengisi slot formulir yang kosong.
    """
    print(f"    > TOOL: ask_user(question='{question}')")
    return question

# Tool 4: Sinyal Konfirmasi (Meta-Tool, Perlu Penyesuaian Nama Argumen)
class ConfirmSchema(BaseModel):
    movie_id: int = Field(description="ID film final.")
    showtime_id: int = Field(description="ID jadwal final.")
    seats: List[str] = Field(description="Daftar kursi final.")
    # --- PERBAIKAN NAMA ARGUMEN ---
    customer_name: str = Field(description="Nama pemesan final.") 
    # --- AKHIR PERBAIKAN ---

@tool(args_schema=ConfirmSchema)
def request_confirmation(movie_id: int, showtime_id: int, seats: List[str], customer_name: str) -> dict:
    """
    Gunakan tool ini HANYA JIKA SEMUA 4 slot (movie_id, showtime_id, seats, customer_name) 
    SUDAH TERISI LENGKAP. Ini adalah sinyal untuk memulai konfirmasi akhir.
    """
    print(f"    > TOOL: request_confirmation(...) dipanggil. Sinyal untuk konfirmasi.")
    return {
        "movie_id": movie_id,
        "showtime_id": showtime_id,
        "seats": seats,
        "customer_name": customer_name 
    }

# --- TOOL MANUAL (Tidak Diekspos ke LLM) ---
def book_tickets_tool(showtime_id: int, seats: List[str], customer_name: str) -> str:
    """Fungsi Python murni untuk eksekusi booking."""
    print(f"    > EKSEKUSI: Mencoba booking {seats} untuk {customer_name} di showtime {showtime_id}")
    # Gunakan nama kolom 'user' saat insert ke DB
    insert_data = [{"showtime_id": showtime_id, "seat": s, "user": customer_name} for s in seats]
    try:
        with engine.connect() as conn:
            with conn.begin():
                # Validasi kursi sebelum insert (Defensive)
                invalid_seats = [s for s in seats if s not in ALL_VALID_SEATS]
                if invalid_seats:
                    raise ValueError(f"Kursi tidak valid ditemukan: {', '.join(invalid_seats)}")
                
                # Cek ketersediaan lagi (Defensive, race condition)
                stmt_check = select(bookings_table.c.seat).where(
                    and_(
                        bookings_table.c.showtime_id == showtime_id,
                        bookings_table.c.seat.in_(seats)
                    )
                )
                already_booked = conn.execute(stmt_check).fetchall()
                if already_booked:
                    booked_list = [r.seat for r in already_booked]
                    raise ValueError(f"Kursi {', '.join(booked_list)} sudah terisi saat mencoba booking.")

                # Insert jika aman
                conn.execute(insert(bookings_table), insert_data)
                
        return f"Sukses! Tiket untuk {customer_name} di kursi {', '.join(seats)} telah dikonfirmasi."
    except ValueError as ve: # Tangkap error validasi kita
        print(f"    > EKSEKUSI GAGAL (Validasi): {ve}")
        return f"Maaf, terjadi masalah: {ve}"
    except Exception as e: # Tangkap error DB (misal UniqueConstraint)
        print(f"    > EKSEKUSI GAGAL (DB): {e}")
        # Coba berikan pesan error yang lebih ramah
        if "uq_booking_showtime_seat" in str(e):
             return f"Maaf, terjadi error saat booking. Salah satu kursi ({', '.join(seats)}) mungkin sudah terisi oleh orang lain."
        return f"Maaf, terjadi error tak terduga saat booking."

## Testing tool (work semua)

In [8]:
#int(get_showtimes.func(2, "2025-10-30")) #OK
print(get_available_seats.func(718)) #O
#rint(ask_user.func("Siapa nama Anda?")) #OK
#rint(request_confirmation.func(2, 718, ["A1", "A2"], "Rafi Wangsa")) #OK

    > TOOL: get_available_seats(showtime_id=718)
    > TOOL get_available_seats: dari 216 kursi, Menemukan 199 tersedia, 17 sudah terisi.
count_available=199 count_booked=17 summary_for_llm='199 kursi tersedia. 17 kursi terisi: A1, A10, A11, A12, F4, F5, F6, F7, F8, F9... Contoh kursi tersedia: A13 ... M9' available_list=['A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F2', 'F3', 'G1', 'G10', 'G11', 'G12', 'G13', 'G1

In [9]:
print(book_tickets_tool(718, ["F4","F5","F6","F7", "F8", "F9"], "Rafi Wangsa")) #OK

    > EKSEKUSI: Mencoba booking ['F4', 'F5', 'F6', 'F7', 'F8', 'F9'] untuk Rafi Wangsa di showtime 718
    > EKSEKUSI GAGAL (Validasi): Kursi F4, F5, F6, F7, F8, F9 sudah terisi saat mencoba booking.
Maaf, terjadi masalah: Kursi F4, F5, F6, F7, F8, F9 sudah terisi saat mencoba booking.


## schemas

In [10]:
# --- GANTI SELURUH CELL "schemas" DENGAN INI ---

# Tools ini adalah 'aksi' untuk MENGISI slot state.
@tool(args_schema=MovieSelectionOutput)
def record_selected_movie(selected_movie_id: Optional[int]) -> str:
    """
    Gunakan ini untuk MENCATAT ID film yang sudah dipilih user.
    Panggil ini SETELAH kamu mencocokkan input user ('Kimi no Nawa') 
    ke ID film dari 'DAFTAR FILM TERSEDIA'.
    Jika user tidak memilih/tidak relevan, panggil dengan 'selected_movie_id: null'.
    """
    if selected_movie_id is None:
        return "OK. Tidak ada film yang dipilih."
    return f"OK. Film ID {selected_movie_id} dicatat."

@tool(args_schema=ShowtimeSelectionOutput)
def record_selected_showtime(selected_showtime_id: Optional[int]) -> str:
    """
    Gunakan ini untuk MENCATAT ID jadwal yang sudah dipilih user.
    Panggil ini SETELAH kamu mencocokkan input user ('jam 7 malam') 
    ke ID jadwal dari 'Jadwal Tersedia'.
    Jika user tidak memilih/tidak relevan, panggil dengan 'selected_showtime_id: null'.
    """
    if selected_showtime_id is None:
        return "OK. Tidak ada jadwal yang dipilih."
    return f"OK. Jadwal ID {selected_showtime_id} dicatat."

@tool(args_schema=SeatsSelectionOutput)
def record_selected_seats(selected_seats_list: Optional[List[str]]) -> str:
    """
    Gunakan ini untuk MENCATAT daftar kursi yang sudah dipilih user.
    Panggil ini SETELAH kamu mengekstrak kursi (['A1', 'A2']) dari input user.
    Jika user tidak memilih/tidak relevan, panggil dengan 'selected_seats_list: null'.
    """
    if not selected_seats_list:
        return "OK. Tidak ada kursi yang dipilih."
    return f"OK. Kursi {', '.join(selected_seats_list)} dicatat."

@tool(args_schema=NameExtractionOutput)
def record_customer_name(extracted_customer_name: Optional[str]) -> str:
    """
    Gunakan ini untuk MENCATAT nama pemesan yang sudah diekstrak.
    Panggil ini SETELAH kamu mengekstrak nama (misal 'Rafi') dari input user.
    Jika user tidak menyebut nama, panggil dengan 'extracted_customer_name: null'.
    """
    if not extracted_customer_name:
        return "OK. Tidak ada nama yang diekstrak."
    return f"OK. Nama {extracted_customer_name} dicatat."

NameError: name 'MovieSelectionOutput' is not defined

## Initisasi model

In [ ]:
# Inisialisasi Model dan Tools
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

## Master Prompt

In [ ]:
def get_simple_master_prompt(state: TicketAgentState) -> List[AnyMessage]:
    """Merakit System Prompt yang simpel."""
    
    # ... (Semua kode untuk mengambil movie_id, showtime_id, movie_list_str, dll. SAMA) ...
    movie_id = state.get('current_movie_id')
    showtime_id = state.get('current_showtime_id')
    seats = state.get('selected_seats')
    customer = state.get('customer_name')
    movie_list_str = "\n".join(
        [f"- ID: {m['id']}, Judul: {m['title']}" for m in state.get("all_movies_list", [])]
    )
    showtime_list = state.get("context_showtimes", [])
    showtime_context_str = "N/A (Panggil 'get_showtimes' dulu)"
    if showtime_list:
        # ... (Logika parse showtime_context_str SAMA) ...
        if isinstance(showtime_list[0], dict) and 'error' in showtime_list[0]:
            showtime_context_str = f"Error: {showtime_list[0]['error']}"
        elif isinstance(showtime_list[0], dict) and 'message' in showtime_list[0]:
            showtime_context_str = showtime_list[0]['message']
        else:
            showtime_context_str = "\n".join(
                [f"- ID: {s['showtime_id']}, Waktu: {s['time_display']}" for s in showtime_list]
            )
    seat_context_str = state.get("context_seats_summary", "N/A (Panggil 'get_available_seats' dulu)")

    prompt_lines = [
        "Anda adalah Manajer Booking. Tugas Anda mengisi formulir langkah demi langkah.",
        "Anda WAJIB memanggil SATU tool yang tersedia. DILARANG KERAS menjawab langsung.",
        "Jika Anda bingung atau butuh info, panggil tool 'ask_user'.",

        f"\n**HISTORI CHAT (5 Terakhir):**\n{state.get('messages', [])[-5:]}",
        f"\n**DAFTAR FILM TERSEDIA:**\n{movie_list_str}",
        f"\n**FORMULIR SAAT INI:**\n"
        f"- current_movie_id: {movie_id or 'BELUM ADA'}\n"
        f"- current_showtime_id: {showtime_id or 'BELUM ADA'}\n"
        f"- selected_seats: {seats or 'BELUM ADA'}\n"
        f"- customer_name: {customer or 'BELUM ADA'}",

        f"\n**KONTEKS TAMBAHAN:**\n"
        f"- Jadwal Tersedia:\n{showtime_context_str}\n"
        f"- Kursi Tersedia: {seat_context_str}\n"
        f"- Error Terakhir: {state.get('last_error') or 'Tidak ada'}",

        "\n**INSTRUKSI LANGKAH SELANJUTNYA:**",
        "1. Tentukan SATU slot formulir BERIKUTNYA yang masih 'BELUM ADA'.",
        "2. Jika slot itu 'current_movie_id': Analisis histori & 'DAFTAR FILM'. Gunakan kognisimu (misal 'Kimi no Nawa' -> 'Your Name') lalu panggil tool `record_selected_movie` dengan ID yang benar. Jika user belum memilih, panggil `ask_user`.",
        "3. Jika slot itu 'current_showtime_id':",
        "   - Jika 'Jadwal Tersedia' == N/A: Cek histori untuk tanggal. Jika ada, panggil `get_showtimes`. Jika tidak, panggil `ask_user` minta tanggal.",
        "   - Jika 'Jadwal Tersedia' SUDAH ADA: Analisis histori. Jika user memilih (misal 'jam 7'), panggil `record_selected_showtime` dengan ID yang benar. Jika belum, panggil `ask_user` minta pilih jadwal.",
        "4. Jika slot itu 'selected_seats':",
        "   - Jika 'Kursi Tersedia' == N/A: Panggil `get_available_seats`.",
        "   - Jika 'Kursi Tersedia' SUDAH ADA: Analisis histori. Jika user memilih (misal 'A1, A2'), panggil `record_selected_seats`. Jika belum, panggil `ask_user` minta pilih kursi.",
        "5. Jika slot itu 'customer_name': Analisis histori. Jika nama ada, panggil `record_customer_name`. Jika belum, panggil `ask_user` minta nama.",
        "6. Jika SEMUA slot terisi: Panggil `request_confirmation`."
    ]
    
    # Gabungkan jadi SystemMessage
    system_prompt = SystemMessage(content="\n".join(prompt_lines))
    return [system_prompt] # Kembalikan HANYA list of messages

## Booking Manager

In [ ]:
# --- GANTI SELURUH BAGIAN Inisialisasi model & Booking Manager DENGAN INI ---

# Gabungkan SEMUA tools (data + aksi + meta)
booking_manager_tools = [
    get_showtimes,
    get_available_seats,
    ask_user,
    request_confirmation,
    record_selected_movie,     # <-- Tool Aksi Baru
    record_selected_showtime,  # <-- Tool Aksi Baru
    record_selected_seats,     # <-- Tool Aksi Baru
    record_customer_name,      # <-- Tool Aksi Baru
]
# Model yang di-bind dengan SEMUA tools
model_with_tools = llm.bind_tools(booking_manager_tools)

# --- Node Utama: Manajer Booking (Versi Simpel & Benar) ---
def node_booking_manager(state: TicketAgentState) -> dict:
    print("--- NODE: Booking Manager ---")
    
    # 1. Dapatkan HANYA system prompt
    system_prompt_list = get_simple_master_prompt(state)
    
    # 2. GABUNGKAN system prompt dengan SEMUA histori chat dari state
    messages_for_llm = system_prompt_list + state.get("messages", [])

    # Siapkan dictionary untuk update state
    current_summary = state.get("context_seats_summary", "N/A")
    updates = {"messages": [], "last_error": None, "context_seats_summary": current_summary} 
    ai_response = None 

    try:
        # 3. SELALU Panggil LLM untuk Tool Call
        print(f"    > Meminta Tool Call...")
        ai_response = model_with_tools.invoke(messages_for_llm)
        print(f"    > Hasil LLM (Tool Call): {ai_response.tool_calls}")
        updates["messages"].append(ai_response) # Tambahkan AIMessage ke state

        if not ai_response.tool_calls:
            print("    > Peringatan: LLM tidak memanggil tool (melanggar instruksi).")
            updates["last_error"] = "LLM gagal memanggil tool."

        # 4. Proses tool call (jika ada) untuk update state
        for tool_call in ai_response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            tool_id = tool_call["id"]
            
            # --- Logika untuk Aksi Pengisian Slot ---
            if tool_name == "record_selected_movie":
                selected_id = tool_args.get("selected_movie_id")
                if selected_id is not None:
                    updates["current_movie_id"] = selected_id
                    # Reset konteks bawahan
                    updates["context_showtimes"] = None; updates["current_showtime_id"] = None;
                    updates["context_seats"] = None; updates["selected_seats"] = None
                    updates["context_seats_summary"] = "N/A"
                # Buat ToolMessage dummy (karena tool ini hanya aksi)
                updates["messages"].append(ToolMessage(content="Movie ID dicatat.", tool_call_id=tool_id))

            elif tool_name == "record_selected_showtime":
                selected_id = tool_args.get("selected_showtime_id")
                if selected_id is not None:
                    updates["current_showtime_id"] = selected_id
                    updates["context_seats"] = None; updates["selected_seats"] = None
                    updates["context_seats_summary"] = "N/A"
                updates["messages"].append(ToolMessage(content="Showtime ID dicatat.", tool_call_id=tool_id))

            elif tool_name == "record_selected_seats":
                seats_list = tool_args.get("selected_seats_list")
                if seats_list:
                    # Validasi (bisa dipindah ke tool-nya jika mau)
                    valid_seats = state.get("context_seats", []) # Ambil dari state
                    if not valid_seats:
                         # Panggil get_available_seats dulu
                         updates["last_error"] = "Agent mencoba rekam kursi sebelum cek ketersediaan."
                         break # Hentikan proses
                         
                    chosen = seats_list
                    validated = [s for s in chosen if s in valid_seats]
                    
                    if len(validated) == len(chosen) and 0 < len(validated) <= 7:
                        updates["selected_seats"] = validated
                    else:
                        print(f"    > Peringatan: LLM memilih kursi tidak valid/kosong/kebanyakan.")
                        updates["last_error"] = "Kursi tidak valid/kosong/lebih dari 7."
                updates["messages"].append(ToolMessage(content="Seats dicatat.", tool_call_id=tool_id))
            
            elif tool_name == "record_customer_name":
                name = tool_args.get("extracted_customer_name")
                if name:
                    updates["customer_name"] = name
                updates["messages"].append(ToolMessage(content="Nama dicatat.", tool_call_id=tool_id))

            # --- Logika untuk Tool Data & Meta ---
            elif tool_name == "get_available_seats":
                 try:
                      seat_info : SeatAvailabilityInfo = get_available_seats.invoke(tool_args)
                      updates["context_seats"] = seat_info.available_list 
                      updates["context_seats_summary"] = seat_info.summary_for_llm 
                      updates["messages"].append(ToolMessage(
                          content=str(seat_info.dict()), 
                          tool_call_id=tool_call["id"]
                      ))
                 except Exception as e:
                      updates["last_error"] = f"Gagal fetch seats: {e}"
                      updates["context_seats_summary"] = "Error ambil kursi."
                      updates["messages"].append(ToolMessage(content=f"Error: {e}", tool_call_id=tool_id))

            # Untuk get_showtimes, ask_user, request_confirmation, kita tidak perlu
            # 'hack' lagi. Router akan menangani pemanggilan tool-nya.
            # Kita hanya perlu pastikan 'model_with_tools' di-invoke.
            # (Router akan memanggil tool-tool ini secara otomatis via LangGraph)
                      
    except Exception as e:
        print(f"    > ERROR saat pemanggilan LLM: {e}")
        updates["last_error"] = f"Gagal memproses langkah: {e}"
        if ai_response is None:
             error_msg = AIMessage(content=f"Maaf, terjadi error internal: {e}")
             updates["messages"].append(error_msg)

    return updates

## Node Konfirm

In [ ]:
def node_confirmation(state: TicketAgentState) -> dict:
    """
    Menampilkan rangkuman final dan mengeksekusi booking.
    Dipicu HANYA setelah tool 'request_confirmation' dipanggil.
    """
    print("--- NODE: Confirmation ---")
    
    # 1. Ambil data final dari state
    # Tool request_confirmation sudah mengisinya di loop sebelumnya
    final_data = state.get("confirmation_data") 
    
    if not final_data:
        print("    > ERROR: Data konfirmasi tidak ditemukan di state!")
        return {"messages": [AIMessage(content="Terjadi error: Data pemesanan tidak lengkap untuk konfirmasi.")]}

    # 2. Ambil detail (Nama film, Waktu tampil) untuk rangkuman
    # (Ini butuh query kecil ke DB)
    movie_title = "(Judul tidak ditemukan)"
    showtime_display = "(Jadwal tidak ditemukan)"
    try:
        with engine.connect() as conn:
            # Ambil judul film
            movie_res = conn.execute(
                select(movies_table.c.title).where(movies_table.c.id == final_data['movie_id'])
            ).first()
            if movie_res:
                movie_title = movie_res.title
                
            # Ambil waktu jadwal (dan konversi ke WIB display)
            showtime_res = conn.execute(
                select(showtimes_table.c.time).where(showtimes_table.c.id == final_data['showtime_id'])
            ).first()
            if showtime_res:
                 showtime_display = from_db_utc_naive_to_local_display(showtime_res.time)

    except Exception as e:
        print(f"    > ERROR saat mengambil detail untuk konfirmasi: {e}")
        # Tetap lanjutkan dengan ID jika detail gagal diambil

    # 3. Buat Rangkuman Teks
    summary = (
        f"✅ **Konfirmasi Pesanan Anda:**\n"
        f"---------------------------\n"
        f"🎬 **Film:** {movie_title} (ID: {final_data['movie_id']})\n"
        f"🗓️ **Jadwal:** {showtime_display} (ID: {final_data['showtime_id']})\n"
        f"💺 **Kursi:** {', '.join(final_data['seats'])}\n"
        f"👤 **Atas Nama:** {final_data['customer_name']}\n"
        f"---------------------------\n"
        f"\n⏳ Memproses pemesanan..."
    )
    # Tampilkan rangkuman ke konsol (opsional)
    print(f"    > Rangkuman:\n{summary}")
    
    # 4. Eksekusi Booking (Panggil fungsi Python biasa)
    result_message = book_tickets_tool(
        showtime_id=final_data['showtime_id'],
        seats=final_data['seats'],
        customer_name=final_data['customer_name'] # Nama argumen sesuai fungsi
    )
    
    print(f"    > Hasil Eksekusi: {result_message}")
    
    # 5. Kembalikan Pesan Final ke User
    final_response = f"{summary}\n\n**Status:** {result_message}"
    
    # Reset state setelah booking (opsional, tapi bagus)
    updates = {
        "messages": [AIMessage(content=final_response)],
        "current_movie_id": None,
        "current_showtime_id": None,
        "selected_seats": None,
        # Biarkan customer_name agar agent ingat? Atau hapus juga?
        # "customer_name": None, 
        "context_showtimes": None,
        "context_seats": None,
        "confirmation_data": None, 
        "last_error": None
    }
    return updates

## Booking Router


In [ ]:
def booking_router(state: TicketAgentState) -> Literal["confirm", "continue", "__end__"]:
    print("--- ROUTER: Booking Router ---")
    
    if state.get("last_error"):
        print(f"    > Rute: __end__ (ERROR terdeteksi: {state.get('last_error')})")
        return "__end__"

    messages = state["messages"]
    last_ai_message = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            last_ai_message = msg
            break

    if not last_ai_message or not last_ai_message.tool_calls:
        # LLM Gagal memanggil tool (melanggar instruksi)
        print("    > Rute: __end__ (NO_TOOL_CALL / LLM bandel)")
        return "__end__"
        
    # Ambil tool call *pertama* (atau terakhir, tergantung logikamu)
    tool_call = last_ai_message.tool_calls[0] 
    tool_name = tool_call["name"]

    if tool_name == "request_confirmation":
        print("    > Rute: confirm (data lengkap)")
        return "confirm"
    elif tool_name == "ask_user":
        print("    > Rute: __end__ (menunggu input user)")
        return "__end__"
    else:
        # Ini adalah tool Aksi (record_...) ATAU tool Data (get_...)
        # Keduanya butuh 'continue' agar LLM bisa memproses hasilnya
        # atau lanjut ke slot berikutnya.
        print(f"    > Rute: continue (Aksi/Tool '{tool_name}' dipanggil, lanjut loop)")
        return "continue"

## Workflow Graph

In [ ]:
# --- 9. Definisi & Kompilasi Graph ---

print("Merakit Graph...")
workflow = StateGraph(TicketAgentState)

# Daftarkan node-node kita
workflow.add_node("booking_manager", node_booking_manager)
workflow.add_node("confirmation", node_confirmation)

# Tentukan titik masuk
workflow.set_entry_point("booking_manager")

# Tambahkan cabang logika utama setelah 'booking_manager'
workflow.add_conditional_edges(
    "booking_manager", # Node sumber
    booking_router,    # Fungsi router yang kita buat
    {
        "confirm": "confirmation",  # Jika router bilang 'confirm', pergi ke node 'confirmation'
        "continue": "booking_manager", # Jika router bilang 'continue', loop kembali ke 'booking_manager'
        "__end__": END               # Jika router bilang '__end__', graph berhenti
    }
)

# Tambahkan cabang akhir setelah konfirmasi (selalu selesai)
workflow.add_edge("confirmation", END)

# Kompilasi graph menjadi aplikasi yang bisa dijalankan
app = workflow.compile()
print("Graph berhasil di-compile.")

# --- 10. Persiapan & Main Chat Loop ---

# (Fungsi get_all_movies_from_db() harus ada di sini atau diimpor)
def get_all_movies_from_db():
    print("Memuat daftar film dari DB...")
    try:
        with engine.connect() as conn:
            # Pastikan kolom title ada
            if 'title' not in movies_table.c:
                 raise KeyError("Kolom 'title' tidak ditemukan di movies_table.")
            rows = conn.execute(select(movies_table.c.id, movies_table.c.title)).fetchall()
            if not rows:
                 print("PERINGATAN: Tidak ada film ditemukan di database.")
            return [{"id": row.id, "title": row.title} for row in rows]
    except Exception as e:
        print(f"ERROR saat memuat daftar film: {e}")
        print("Menggunakan daftar film contoh sebagai fallback.")
        # Fallback jika DB error atau kosong
        return [
            {'id': 1, 'title': 'Spirited Away'}, 
            {'id': 2, 'title': 'Your Name'},
            {'id': 3, 'title': 'Attack on Titan: Requiem'} 
        ]


ALL_MOVIES_CONTEXT = get_all_movies_from_db()
print(f"Total {len(ALL_MOVIES_CONTEXT)} film dimuat ke konteks.")
# Cetak daftar film untuk debug (opsional)
# print(ALL_MOVIES_CONTEXT) 

# State awal untuk setiap sesi baru
INITIAL_STATE: TicketAgentState = {
    "messages": [],
    "intent": "booking", # Kita asumsikan selalu mulai dengan niat booking
    "all_movies_list": ALL_MOVIES_CONTEXT,
    "current_movie_id": None,
    "current_showtime_id": None,
    "selected_seats": None,
    "customer_name": None,
    "context_showtimes": None,
    "context_seats": None, # List lengkap kursi
    "context_seats_summary": "N/A", # Ringkasan kursi untuk prompt
    "confirmation_data": None,
    "last_error": None,
}

# Penyimpanan state antar giliran (dalam memori, ganti dengan DB/Redis untuk production)
session_states = {}
SESSION_ID = "user_session_123" # ID sesi unik per user

print("\n--- Agen Manajer Booking Siap! ---")
print("Ketik 'exit' untuk keluar.")
print("Contoh: 'mau pesan tiket', 'kimi no nawa', '2025-10-30', 'A1', 'Rafi'")

Merakit Graph...
Graph berhasil di-compile.
Memuat daftar film dari DB...


Total 21 film dimuat ke konteks.

--- Agen Manajer Booking Siap! ---
Ketik 'exit' untuk keluar.
Contoh: 'mau pesan tiket', 'kimi no nawa', '2025-10-30', 'A1', 'Rafi'


## Run

In [ ]:
while True:
    try:
        user_input = input("\nAnda: ")
        if user_input.lower() == "exit":
            break

        # Ambil state sesi ini atau buat baru
        current_state = session_states.get(SESSION_ID, deepcopy(INITIAL_STATE))

        # Tambahkan pesan user ke state
        current_state["messages"].append(HumanMessage(content=user_input))

        # --- Panggil Graph ---
        print("\nAgen:")
        # Gunakan stream untuk melihat proses internal (opsional tapi bagus untuk debug)
        # final_state = None
        # for event in app.stream(current_state, {"recursion_limit": 50}):
        #     # Print event jika perlu (misal: event keys, values)
        #     # print(event) 
        #     # Ambil state terakhir dari event
        #     last_key = list(event.keys())[-1]
        #     final_state = event[last_key]
        
        # Atau pakai invoke untuk lebih simpel
        final_state = app.invoke(current_state, {"recursion_limit": 50})
        # --- Akhir Panggilan Graph ---
            
        if final_state is None:
             print("Error: Graph tidak menghasilkan state akhir.")
             continue

        # Simpan state baru untuk giliran berikutnya
        session_states[SESSION_ID] = final_state

        # --- Logika Menampilkan Output ke User ---
        # Ambil pesan AI terakhir (setelah semua proses node & tool)
        agent_messages = final_state.get("messages", [])
        last_agent_message = agent_messages[-1] if agent_messages else None

        output_to_show = "(Agen tidak memberikan respons)" # Default

        if isinstance(last_agent_message, AIMessage):
            if last_agent_message.tool_calls:
                # Cek apakah tool call terakhir adalah ask_user
                if last_agent_message.tool_calls[-1]["name"] == "ask_user":
                    output_to_show = last_agent_message.tool_calls[-1]["args"]["question"]
                else:
                    # Untuk tool lain, kita mungkin tidak perlu bilang apa-apa
                    # Atau beri pesan generik
                     output_to_show = "(Agen sedang memproses...)" 
            elif last_agent_message.content:
                # Tampilkan content jika tidak ada tool call (jawaban/konfirmasi final)
                output_to_show = last_agent_message.content
        elif isinstance(last_agent_message, ToolMessage):
             # Jika berakhir di ToolMessage (seharusnya tidak terjadi, tapi jaga-jaga)
             output_to_show = f"(Debug: Proses berakhir dengan hasil tool {last_agent_message.name})"
        
        print(output_to_show)
        # --- Akhir Logika Tampilan ---

    except KeyboardInterrupt:
        print("\nBerhenti...")
        break
    except Exception as e:
        print(f"\nFATAL ERROR: Terjadi error tidak terduga: {e}")
        import traceback
        traceback.print_exc()
        # Mungkin reset state sesi ini jika terjadi error?
        # if SESSION_ID in session_states: del session_states[SESSION_ID] 
        break

print("\nSesi Selesai.")

# Cek Isian

In [ ]:
print(ALL_MOVIES_CONTEXT)

[{'id': 1, 'title': 'Spirited Away'}, {'id': 2, 'title': 'Your Name'}, {'id': 3, 'title': 'Akira'}, {'id': 4, 'title': 'Ghost in the Shell'}, {'id': 5, 'title': 'Attack on Titan: Requiem'}, {'id': 6, 'title': 'Mobile Suit Gundam SEED Freedom'}, {'id': 7, 'title': 'Interstellar'}, {'id': 8, 'title': 'Blade Runner 2049'}, {'id': 9, 'title': 'Dune: Part Two'}, {'id': 10, 'title': 'The Matrix'}, {'id': 11, 'title': 'Star Wars: A New Hope'}, {'id': 12, 'title': 'Mad Max: Fury Road'}, {'id': 13, 'title': 'Everything Everywhere All at Once'}, {'id': 14, 'title': 'Black Panther'}, {'id': 15, 'title': 'The Dark Knight'}, {'id': 16, 'title': 'Inception'}, {'id': 17, 'title': 'Pulp Fiction'}, {'id': 18, 'title': 'Parasite'}, {'id': 19, 'title': 'The Lord of the Rings: The Fellowship of the Ring'}, {'id': 20, 'title': 'Coco'}, {'id': 21, 'title': 'La La Land'}]
